<a href="https://colab.research.google.com/github/ucheokechukwu/ml_tensorflow_deeplearning/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning With TF part 3: Scaling Up (Food Vision Mini 🍖🕶)

Our goal is to beat the original Food101 Paper with 10% of the training data, leveraging transfer learning.

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-22 05:58:24--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-22 05:58:24 (92.3 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [3]:
from helper_functions import walk_through_dir, unzip_data, plot_loss_curves, compare_historys

## 101 Food Classes: working with less data

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2023-02-22 05:58:30--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.128, 173.194.213.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10 100%[===================>]   1.51G   155MB/s    in 13s     

2023-02-22 05:58:44 (116 MB/s) - ‘101_food_classes_10_percent.zip.1’ saved [1625420029/1625420029]



In [5]:
unzip_data("/content/101_food_classes_10_percent.zip")

In [6]:
walk_through_dir("/content/101_food_classes_10_percent")

There are 2 directories and 0 images in '/content/101_food_classes_10_percent'.
There are 101 directories and 0 images in '/content/101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/cannoli'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/edamame'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/chocolate_cake'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/pancakes'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/breakfast_burrito'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/beet_salad'.
There are 0 directories and 250 images in '/content/101_food_classes_10_percent/test/gnocchi'.
There are 0 directories and 250 images in '/c

In [7]:
# Set up data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = "/content/101_food_classes_10_percent/train/"
test_dir = "/content/101_food_classes_10_percent/test/"

train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE,
                                                                                batch_size=BATCH_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                batch_size=BATCH_SIZE,
                                                                shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


Steps:
1. Create a ModelCheckpoint callback
2. Creat a data augmentation layer to build data augmentation right into the model
3. Build a headless (no top layers) Functional EfficientNetB60 backboned-model (we'll create our own output layer).
4. Compile our model
5. Feature extract for 5 full epochs on train dataset and validate on 15% of the test data, to save epoch time.

In [8]:
# Create ModelCheckpoint 
checkpoint_path="101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [10]:
# Create data augmentation layer
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing
data_augmentation = tf.keras.Sequential([
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomFlip("horizontal")
    
], name="data_augmentation")

In [13]:
# Create a headless functional EfficientNet60 backboned model

inputs = layers.Input(shape=(224,224,3), name="input_layer")
x = data_augmentation(inputs)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False
x = base_model(x)
x = layers.GlobalAveragePooling2D(name="global_average_pooling_2d")(x)
outputs = layers.Dense(101, activation="softmax")(x)
model_ = tf.keras.models.Model(inputs, outputs)

In [14]:
model_.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, None, None, 3)    0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling_2d (  (None, 1280)             0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense_1 (Dense)             (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-train